In [ ]:
import manganite
%load_ext manganite

# Portfolio Selection Optimization
This model is an example of the classic [Markowitz portfolio selection optimization model](https://en.wikipedia.org/wiki/Markowitz_model). We want to find the fraction of the portfolio to invest among a set of stocks that balances risk and return. It is a Quadratic Programming (QP) model with vector and matrix data for returns and risk, respectively. This is best suited to a matrix formulation, so we use the Gurobi Python *matrix* interface. The basic model is fairly simple, so we also solve it parametrically to find the efficient frontier.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

**Gurobi License** <br /> 
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-MUI-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_Lost_Luggage_Distribution_COM_EVAL_GitHub&utm_term=Lost%20Luggage%20Distribution&utm_content=C_JPM) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-EDU-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_Lost_Luggage_Distribution_COM_EVAL_GitHub&utm_term=Lost%20Luggage%20Distribution&utm_content=C_JPM) as an *academic user*.


## Model Formulation
### Parameters

We use the [Greek values](https://en.wikipedia.org/wiki/Greeks_\(finance\)) that are traditional in finance:

- $$ \delta $$: n-element vector measuring the change in price for each stock
- $$ \sigma $$: n x n matrix measuring the covariance among stocks

There is one additional parameter when solving the model parametrically:

- r: target return


### Decision Variables
- $$ x \ge 0 $$ : n-element vector where each element represents the fraction of the porfolio to invest in each stock

### Objective Function
Minimize the total risk, a convex quadratic function:
<pre>
$$
\begin{equation}
\min x^t \cdot \sigma \cdot x
\end{equation}
$$
</pre>

### Constraints

Allocate the entire portfolio: the total investments should be 1.0 (100%), where $e$ is a unit vector (all 1's):
<pre>
$$
\begin{equation}
e \cdot x = 1
\end{equation}
$$
</pre>
Return: When we solve the model parametrically for different return values $r$, we add a constraint on the target return:
<pre>
$$
\begin{equation}
\delta \cdot x = r
\end{equation}
$$
</pre>

## Python Implementation
### Stock data
Use [yfinance](https://pypi.org/project/yfinance/) library to get the latest 2 years of _actual stock data_ from the 20 most profitable US companies, [according to Wikipedia in April 2021](https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue#List_of_companies_by_profit).
### Dashboard
Use manganite package to create a beautiful dashoard from the jupyter notebook

In [ ]:
import yfinance as yf
import numpy as np

import gurobipy as gp
from gurobipy import GRB
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import plotly.io as pio
pio.templates.default = 'plotly_white'

In [ ]:
%%mnn widget --type text --tab "Stock Selector" --header "Select your stocks" --var stocks --position 0 0 3
stocks = 'AAPL, MSFT, JPM, GOOG, BAC, INTC, WFC, Meta'

In [ ]:
# Split the input string into a list using the column separator
tick_marks_list = stocks.split(',')

# Remove unwanted characters and spaces from each element
clean_tick_marks = [tick.strip() for tick in tick_marks_list]

data = yf.download(clean_tick_marks, period='2y')

In [ ]:
%%mnn widget --type plot --var fig_stocks --tab "Stock Selector" --position 1 0 4 --header "Stock prices"

df_closing = data['Close']
fig_stocks = px.line(df_closing, x=df_closing.index, y=df_closing.columns,
            #   hover_data={"date": "|%B %d, %Y"},
              )

# Update the layout to customize axis labels
fig_stocks.update_yaxes(title_text='Stock Price ($)')

fig_stocks.update_xaxes(
  title_text='',
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")

fig_stocks.update_layout(legend_title="Stock",)

In [ ]:
%%mnn execute --on button "Optimize Portfolio" --returns solution

#calculating greeks
closes = np.transpose(np.array(data.Close)) # matrix of daily closing prices
absdiff = np.diff(closes)                   # change in closing price each day
reldiff = np.divide(absdiff, closes[:,:-1]) # relative change in daily closing price
delta = np.mean(reldiff, axis=1)            # mean price change
sigma = np.cov(reldiff)                     # covariance (standard deviations)
std = np.std(reldiff, axis=1)               # standard deviation
df_plot = pd.DataFrame({'std': std, 'delta': delta, 'stocks':clean_tick_marks})
print('solving QP model')
# Create an empty model
m = gp.Model('portfolio')

# Add matrix variable for the stocks
x = m.addMVar(len(clean_tick_marks))

# Objective is to minimize risk (squared).  This is modeled using the
# covariance matrix, which measures the historical correlation between stocks
portfolio_risk = x @ sigma @ x
m.setObjective(portfolio_risk, GRB.MINIMIZE)

# Fix budget with a constraint
m.addConstr(x.sum() == 1, 'budget')

# Verify model formulation
# m.write('portfolio_selection_optimization.lp')

# Optimize model to find the minimum risk portfolio
m.optimize()

minrisk_volatility = sqrt(m.ObjVal)
minrisk_return = delta @ x.X

# Create an expression representing the expected return for the portfolio
portfolio_return = delta @ x

target = m.addConstr(portfolio_return == minrisk_return, 'target')

# solution = pd.DataFrame(data=np.append(x.X, [minrisk_volatility, minrisk_return]),
#              index=clean_tick_marks + ['Volatility', 'Expected Return'],
#              columns=['Minimum Risk Portfolio'])

solution = pd.DataFrame(data=x.X,
             index=clean_tick_marks,
             columns=['Minimum Risk Portfolio'])

exp_return = minrisk_return
exp_volatility = minrisk_volatility

# Solve for efficient frontier by varying target return
frontier = np.empty((2,0))
for r in np.linspace(delta.min(), delta.max(), 25):
    target.rhs = r
    m.optimize()
    frontier = np.append(frontier, [[sqrt(m.ObjVal)],[r]], axis=1)



In [ ]:
%%mnn widget --type plot --var fig_bar --tab "Portfolio" --position 0 0 3 --header "Minimum Risk Portfolio"
# Create a pie chart
fig_pie = px.pie(solution, values='Minimum Risk Portfolio', names=solution.index,
        title=f'Your Portfolio Distribution<br>Expected Return: {round(exp_return,6)}<br>Volatility: {round(exp_volatility,4)}')
fig_pie.update_traces(textposition='inside', textinfo='percent+label')

# Create a bar chart
fig_bar = px.bar(solution, x=solution.index, y='Minimum Risk Portfolio',
title=f'Your Portfolio Distribution<br>Expected Return: {round(exp_return,6)}<br>Volatility: {round(exp_volatility,4)}',
)

# Hide x-axis and y-axis labels
fig_bar.update_xaxes(title_text='')
fig_bar.update_yaxes(title_text='')

In [ ]:
%%mnn widget --type plot --var fig --tab "Portfolio" --position 0 3 3 --header "Efficient Frontier"

update = solution
# Plot volatility versus expected return for individual stocks
fig1 = px.scatter(df_plot, x="std", y="delta" ,
                                labels = { "std": "Volatility (standard deviation)", "delta": "Expected Return"}, text="stocks" )
fig1.update_traces(textposition="bottom right")

# Plot volatility versus expected return for minimum risk portfolio

fig2 = px.scatter(x=[minrisk_volatility], y=[minrisk_return],text = ["Minimum Risk<br>Portfolio"])
fig2.update_traces(textposition="bottom right")
fig  = go.Figure(data=fig1.data + fig2.data)

# Plot efficient frontier

fig.add_trace(go.Scatter(x=frontier[0], y=frontier[1], mode='lines', name='Efficient Frontier'))

# Set x and y labels using update_xaxes and update_yaxes
fig.update_xaxes(title_text="Volatility (standard deviation)")
fig.update_yaxes(title_text="Expected Return")

fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="right",
    x=0.99
))

